<a href="https://colab.research.google.com/github/samirmaalouf/ffmpeg-GUI/blob/main/FFmpeg_GUI_for_VS_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk
import subprocess
import threading
import os  # Import os for path handling

class FFmpegGUI:
    def __init__(self, master):
        self.master = master
        master.title("FFmpeg GUI")

        # Input File
        self.input_label = tk.Label(master, text="Input File:")
        self.input_label.grid(row=0, column=0, sticky="w", padx=5, pady=5)

        self.input_entry = tk.Entry(master, width=50)
        self.input_entry.grid(row=0, column=1, columnspan=2, padx=5, pady=5)

        self.input_button = tk.Button(master, text="Browse", command=self.browse_input)
        self.input_button.grid(row=0, column=3, padx=5, pady=5)

        # Output File
        self.output_label = tk.Label(master, text="Output File:")
        self.output_label.grid(row=1, column=0, sticky="w", padx=5, pady=5)

        self.output_entry = tk.Entry(master, width=50)
        self.output_entry.grid(row=1, column=1, columnspan=2, padx=5, pady=5)

        self.output_button = tk.Button(master, text="Browse", command=self.browse_output)
        self.output_button.grid(row=1, column=3, padx=5, pady=5)

        # FFmpeg Options
        self.options_label = tk.Label(master, text="FFmpeg Options:")
        self.options_label.grid(row=2, column=0, sticky="w", padx=5, pady=5)

        self.options_entry = tk.Entry(master, width=50)
        self.options_entry.grid(row=2, column=1, columnspan=2, padx=5, pady=5)

        # Convert Button
        self.convert_button = tk.Button(master, text="Convert", command=self.convert)
        self.convert_button.grid(row=3, column=1, pady=10)

        # Progress Bar
        self.progress = ttk.Progressbar(master, orient="horizontal", mode="indeterminate")
        self.progress.grid(row=4, column=0, columnspan=4, pady=10, sticky="ew", padx=5)

        # Log Text
        self.log_text = tk.Text(master, height=10, width=60)
        self.log_text.grid(row=5, column=0, columnspan=4, pady=10, padx=5, sticky="nsew")

        self.scrollbar = ttk.Scrollbar(master, command=self.log_text.yview)
        self.scrollbar.grid(row=5, column=4, sticky='ns')

        self.log_text.config(yscrollcommand=self.scrollbar.set)

        self.running_process = None
        master.grid_columnconfigure(1, weight=1) #make the entry widgets expand

    def browse_input(self):
        filename = filedialog.askopenfilename()
        if filename: #check if a file was selected.
            self.input_entry.delete(0, tk.END)
            self.input_entry.insert(0, filename)

    def browse_output(self):
        filename = filedialog.asksaveasfilename()
        if filename: #check if a file was selected.
            self.output_entry.delete(0, tk.END)
            self.output_entry.insert(0, filename)

    def convert(self):
        input_file = self.input_entry.get()
        output_file = self.output_entry.get()
        options = self.options_entry.get()

        if not input_file or not output_file:
            messagebox.showerror("Error", "Please select input and output files.")
            return

        # Construct the FFmpeg command.  Use shell=False for better security and
        # cross-platform compatibility.  Use a list for the command.
        command = ["ffmpeg", "-i", input_file]
        if options:
            command.extend(options.split())  # Split options string into a list
        command.append(output_file)

        self.log_text.delete(1.0, tk.END)
        self.progress.start()
        self.convert_button["state"] = "disabled"

        def run_ffmpeg():
            try:
                process = subprocess.Popen(command, stderr=subprocess.PIPE, text=True)
                self.running_process = process
                for line in process.stderr:
                    self.log_text.insert(tk.END, line)
                    self.log_text.see(tk.END)
                process.wait()

                if process.returncode == 0:
                    messagebox.showinfo("Success", "Conversion complete!")
                else:
                    messagebox.showerror("Error", f"FFmpeg failed with return code: {process.returncode}")

            except FileNotFoundError:
                messagebox.showerror("Error", "FFmpeg not found. Make sure it's installed and in your PATH.")

            except Exception as e:
                messagebox.showerror("Error", f"An error occurred: {e}")

            finally:
                self.progress.stop()
                self.convert_button["state"] = "normal"
                self.running_process = None

        threading.Thread(target=run_ffmpeg).start()

def main():
    root = tk.Tk()
    gui = FFmpegGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()

TclError: no display name and no $DISPLAY environment variable